In [1]:
%cd ..

/home/vagrant/Python/projects/ashrae-energy-prediction


In [16]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import ruptures as rpt

import json
import numpy as np
import pandas as pd
import sys
sys.path.append(".")

from scripts.models import get_model_cls
from scripts.runner import Runner
from scripts.data.loader import load_datasets

from sklearn.preprocessing import StandardScaler

In [17]:
train = pd.read_feather("data/processed/train.ftr")
test = pd.read_feather("data/processed/test.ftr")

In [18]:
train = train.assign(segment=pd.read_pickle("features/segment_train.pkl")).assign(segment_f=pd.read_pickle("features/segment_fine_train.pkl"))

In [19]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,wind_direction,wind_speed,hour,day,month,dayofweek,meter_label,log_meter_reading,segment,segment_f
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,25.0,...,0.0,0.0,0,1,1,4,electricity,0.0,0.0,0.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,25.0,...,0.0,0.0,0,1,1,4,electricity,0.0,0.0,0.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,25.0,...,0.0,0.0,0,1,1,4,electricity,0.0,0.0,0.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,25.0,...,0.0,0.0,0,1,1,4,electricity,0.0,0.0,0.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,25.0,...,0.0,0.0,0,1,1,4,electricity,0.0,0.0,0.0


In [35]:
bid = 501
b = train.query(f"building_id == {bid}")
b["segment"] = b["segment"] * b["meter_reading"].mean().mean()
b["segment_f"] = b["segment_f"] * b["meter_reading"].mean().mean()
b.head()

/home/vagrant/.local/share/virtualenvs/ashrae-energy-prediction-jiESI2U-/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/vagrant/.local/share/virtualenvs/ashrae-energy-prediction-jiESI2U-/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,wind_direction,wind_speed,hour,day,month,dayofweek,meter_label,log_meter_reading,segment,segment_f
660,501,0,2016-01-01 00:00:00,64.620003,3,Education,66404,NaN,NaN,10.0,...,350.0,4.1,0,1,1,4,electricity,4.183880,0.0,0.0
2959,501,0,2016-01-01 01:00:00,62.549999,3,Education,66404,NaN,NaN,9.4,...,350.0,6.2,1,1,1,4,electricity,4.151827,0.0,0.0
5254,501,0,2016-01-01 02:00:00,62.889999,3,Education,66404,NaN,NaN,8.9,...,10.0,4.6,2,1,1,4,electricity,4.157163,0.0,0.0
7552,501,0,2016-01-01 03:00:00,72.129997,3,Education,66404,NaN,NaN,7.8,...,350.0,7.7,3,1,1,4,electricity,4.292239,0.0,0.0
9847,501,0,2016-01-01 04:00:00,67.570000,3,Education,66404,NaN,NaN,7.8,...,340.0,4.1,4,1,1,4,electricity,4.227855,0.0,0.0


In [36]:
px.line(
    b.melt(id_vars=["timestamp", "meter_label"], value_vars=["meter_reading", "segment", "segment_f"]),
    x="timestamp", y="value", color="meter_label", line_dash="variable"
)